In [1]:
import os, json, pprint, csv, pickle, elasticsearch, time, sys

In [3]:
# 1 entity name-uri mapping
name_to_uri = json.load(open('/home/rovera/cm/scripts/cooccurrence/output/name_to_uri.json', 'r', encoding="utf-8"))
uri_to_name = json.load(open('/home/rovera/cm/scripts/cooccurrence/output/uri_to_name.json', 'r', encoding="utf-8"))
# 2 entity pages inverted index
ep_inv_index = pickle.load(open('/home/rovera/jl/scripts/entity_pages/ep/runs/run_0/ep_inv_index.pickle', 'rb'))
# 3 occurrence by journal
occurrence_by_journal = pickle.load(open('/home/rovera/cm/scripts/cooccurrence/output/occ_by_journal_detail.pickle', 'rb'))
# 4 related entities
entity_correlation = pickle.load(open('/home/rovera/cm/scripts/cooccurrence/output/entity_correlation.pickle', 'rb'))
# 5 data types 
classified_entities = json.load(open('/home/rovera/cm/scripts/jl_linking/classified_entities.json', 'r', encoding="utf-8"))
# 6 TAGME mentions
#cm_entities = pickle.load(open("/home/rovera/cm/scripts/cooccurrence/cm_tagme_resource_reference_data_05_03.pickle", 'rb'))

In [4]:
data = []
i,j,chunk = 0,0,0
for name, uri in name_to_uri.items():
    i += 1
    j += 1
    occs_by_journal = []
    for journal_info, mentions in occurrence_by_journal[uri].items():
        journal_mentions = {'j_name': journal_info[1],
                                'j_id': journal_info[0],
                                'first': mentions['first'],
                                'last': mentions['last'],
                               'mentions': mentions['data']
                               }
        occs_by_journal.append(journal_mentions)


    occs_by_journal = sorted(occs_by_journal, key=lambda x:len(x['mentions']), reverse=True)
    related_entities = sorted([[res_uri, uri_to_name[res_uri], score] for res_uri, score in entity_correlation[uri].items()], key=lambda x:x[2], reverse=True)

    # add entity type information to related entities
    for rel_ent in related_entities:
        ent_type = classified_entities[rel_ent[1].replace(' ', '_')]
        rel_ent.append(ent_type)


    label_name = name.replace(' ', '_')
    e_type = classified_entities[label_name]
    if len(e_type) == 0:
        e_type = "OTH"


    header = {'_id': str(j), '_index': 'cm_entities'}
    body = {'name': name,
            'e_type': e_type,
            'journal_occs': occs_by_journal,
            'related_entities': related_entities,
            'ep': ''
           }
    # to be fixed: not all entity pages exist!
    if uri in ep_inv_index:
        body['ep'] = ep_inv_index[uri]


    data.append(json.dumps({"index": header}, ensure_ascii=False))
    data.append(json.dumps(body, ensure_ascii=False))

    if i == 200:
        print(sys.getsizeof(data))
        with open('/data/cm/output/temp_journals_detail/chunk_'+str(chunk)+'.json', "w", encoding="utf-8") as write_file:
            write_file.write("\n".join(data))
        data = []
        i = 0
        chunk += 1
        print("chunk", chunk, end=" ")

    
with open('/data/cm/output/temp_journals_detail/chunk_'+str(chunk)+'.json', "w", encoding="utf-8") as write_file:
    write_file.write("\n".join(data))
print(i)

3296
chunk 1 3296
chunk 2 3296
chunk 3 3296
chunk 4 3296
chunk 5 3296
chunk 6 3296
chunk 7 3296
chunk 8 3296
chunk 9 3296
chunk 10 3296
chunk 11 3296
chunk 12 3296
chunk 13 3296
chunk 14 3296
chunk 15 3296
chunk 16 3296
chunk 17 3296
chunk 18 3296
chunk 19 3296
chunk 20 3296
chunk 21 3296
chunk 22 3296
chunk 23 3296
chunk 24 3296
chunk 25 3296
chunk 26 3296
chunk 27 3296
chunk 28 3296
chunk 29 3296
chunk 30 54


In [5]:
#define mappings
mappings = {
    "properties": {
        'name': {'type': 'text'},
        'ep': {'type': 'text'},
        'e_type': {'type': 'text'},
        'journal_occs': {'type': 'nested',
                        'properties': {
                            'j_name': {'type': 'text'},
                            'j_id': {'type': 'text'},
                            'first': {'type': 'integer'},
                            'last': {'type': 'integer'},
                            'mentions': {
                                'type': 'nested',
                                'properties': {
                                    'p_id': {'type': 'text'},
                                    'spot': {'type': 'text'},
                                    'start': {'type': 'integer'},
                                    'end': {'type': 'integer'},
                                    'p_link': {'type': 'text'},
                                    'date': {'type': 'text'}
                                } 
                            }
                        }
                        },
        'related_entities': {'type': 'text'}
    }
}

In [6]:
# Create ES instance
es = elasticsearch.Elasticsearch()
ic = elasticsearch.client.IndicesClient(es)
if ic.exists('cm_entities'):
    ic.delete('cm_entities')
ic.create(index='cm_entities', body={'settings': {"index.mapping.nested_objects.limit": 500000}, 'mappings': mappings})

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'cm_entities'}

In [7]:
# Indexing data
for file in os.listdir('/data/cm/output/temp_journals_detail/'):
    if file != '.ipynb_checkpoints':
        print(file)

        data = open('/data/cm/output/temp_journals_detail/'+file, "r", encoding="utf-8").read()

        es_index = es.bulk(data, request_timeout=200)

chunk_30.json
chunk_12.json
chunk_2.json
chunk_28.json
chunk_22.json
chunk_14.json
chunk_18.json
chunk_25.json
chunk_5.json
chunk_6.json
chunk_4.json
chunk_24.json
chunk_13.json
chunk_16.json
chunk_21.json
chunk_29.json
chunk_11.json
chunk_0.json
chunk_10.json
chunk_1.json
chunk_17.json
chunk_20.json
chunk_8.json
chunk_3.json
chunk_26.json
chunk_7.json
chunk_19.json
chunk_23.json
chunk_27.json
chunk_9.json
chunk_15.json


In [8]:
# test query
es = elasticsearch.Elasticsearch()
start = time.time()
res = es.search(index='cm_entities', body={'size': 1, "query": {"match_phrase": {'name': 'Israel'}}})
end = time.time()
print("Search took", round(end-start, 2))

Search took 3.36


In [9]:
print("Total hits:", res['hits']['total']['value'])
for doc in res['hits']['hits']:
    print(doc['_source']['name'])

Total hits: 30
Israel
